<a href="https://colab.research.google.com/github/timosachsenberg/pyopenms-idfreeqc/blob/main/hackathon_ID_free_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyOpenMS ID-Free QC Demo

This notebook demonstrates how to use the `pyopenms-idfreeqc` library to calculate quality control metrics from mzML files.

## Setup

First, let's import the necessary libraries and download demo data.

In [ ]:
# Remove existing directory for a clean setup
!rm -rf pyopenms-idfreeqc

# Clone the repository
!git clone https://github.com/timosachsenberg/pyopenms-idfreeqc.git

# Change into the cloned directory
%cd pyopenms-idfreeqc

# Install uv (if not already globally available)
!pip install uv

# Create and sync the virtual environment using uv
# This ensures all dependencies, including pyopenms, are installed into .venv
!uv sync --all-extras --dev

import sys, os

# Determine the path to the virtual environment's site-packages
# Python version is 3.12 based on the initial output.
python_version_str = f"python{sys.version_info.major}.{sys.version_info.minor}"
venv_site_packages_path = os.path.join(os.getcwd(), '.venv', 'lib', python_version_str, 'site-packages')

# Add the virtual environment's site-packages to sys.path
# This makes packages installed in the venv available to the notebook kernel.
if os.path.exists(venv_site_packages_path) and venv_site_packages_path not in sys.path:
    sys.path.insert(0, venv_site_packages_path)
    print(f"Added virtual environment site-packages to sys.path: {venv_site_packages_path}")
else:
    print(f"Virtual environment site-packages not added to sys.path. Path exists: {os.path.exists(venv_site_packages_path)}, Already in sys.path: {venv_site_packages_path in sys.path}")

# Add the 'src' directory of the pyopenms-idfreeqc library to sys.path
# This is necessary for importing local modules like pyopenms_idfreeqc.calculate_metrics
project_src_path = os.path.abspath("src")
if project_src_path not in sys.path:
    sys.path.append(project_src_path)
    print(f"Added project src path to sys.path: {project_src_path}")
else:
    print(f"Project src path already in sys.path: {project_src_path}")


# Now perform the imports
import json
from urllib.request import urlretrieve
from pyopenms_idfreeqc.calculate_metrics import calculate_metrics, print_metrics_tables
from IPython.display import Image, display, JSON

Cloning into 'pyopenms-idfreeqc'...
remote: Enumerating objects: 183, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 183 (delta 44), reused 55 (delta 25), pack-reused 78 (from 1)
Receiving objects: 100% (183/183), 5.29 MiB | 24.06 MiB/s, done.
Resolving deltas: 100% (83/83), done.
/content/pyopenms-idfreeqc/pyopenms-idfreeqc/pyopenms-idfreeqc/pyopenms-idfreeqc/pyopenms-idfreeqc
Using CPython 3.12.12 interpreter at: /usr/bin/python3
Creating virtual environment at: .venv
Resolved 51 packages in 1ms
Prepared 1 package in 377ms
Installed 36 packages in 120ms
 + attrs==25.4.0
 + certifi==2025.10.5
 + chardet==5.2.0
 + charset-normalizer==3.4.4
 + click==8.3.0
 + contourpy==1.3.3
 + cycler==0.12.1
 + fastobo==0.14.1
 + fonttools==4.60.1
 + idna==3.11
 + jsonschema==4.25.1
 + jsonschema-specifications==2025.9.1
 + kiwisolver==1.4.9
 + matplotlib==3.10.7
 + networkx==3.5
 + numpy==2.3.4
 + packaging==25.0
 + pandas==2.3.3
 

## Download Demo Data

We'll download a few example mzML files for demonstration.

In [ ]:
# Demo mzML files (mix of proteomics and metabolomics)
DEMO_FILES = {
    "https://raw.githubusercontent.com/OpenMS/OpenMS/refs/heads/develop/share/OpenMS/examples/BSA/BSA1.mzML": "BSA1.mzML",
    "https://raw.githubusercontent.com/OpenMS/OpenMS/refs/heads/develop/share/OpenMS/examples/BSA/BSA2.mzML": "BSA2.mzML",
    "https://raw.githubusercontent.com/OpenMS/OpenMS/refs/heads/develop/share/OpenMS/examples/BSA/BSA3.mzML": "BSA3.mzML",
}

# Download files if they don't exist
demo_dir = "demo_data"
os.makedirs(demo_dir, exist_ok=True)

demo_file_paths = []
for url, filename in DEMO_FILES.items():
    filepath = os.path.join(demo_dir, filename)
    demo_file_paths.append(filepath)

    if os.path.exists(filepath):
        print(f"✓ {filename} already exists")
    else:
        print(f"Downloading {filename}...")
        urlretrieve(url, filepath)
        print(f"✓ Downloaded {filename}")

print(f"\nReady to process {len(demo_file_paths)} files")

✓ Downloaded BSA1.mzML
✓ Downloaded BSA2.mzML
✓ Downloaded BSA3.mzML

Ready to process 3 files


## Calculate QC Metrics

Now let's calculate the metrics using the library's main function.

In [ ]:
# Define output paths
output_json = "demo_qc_metrics.mzQC.json"
output_plot = "demo_qc_heatmap.png"

# Calculate metrics
# Note: show_tables=False to suppress table output here (we'll format it nicely later)
# show_json=False to avoid printing raw JSON in notebook output
json_output = calculate_metrics(
    mzml_files=demo_file_paths,
    output_file=output_json,
    generate_plot=True,
    plot_output=output_plot,
    show_tables=False,  # We'll display tables separately
    show_json=False     # We'll display JSON nicely using IPython
)

print("\n✅ Metrics calculation complete!")

Processing mzML files and computing QC metrics...

Processing demo_data/BSA1.mzML...
  ✓ Computed 96 QC metrics for demo_data/BSA1.mzML

Processing demo_data/BSA2.mzML...
  ✓ Computed 96 QC metrics for demo_data/BSA2.mzML

Processing demo_data/BSA3.mzML...
  ✓ Computed 96 QC metrics for demo_data/BSA3.mzML

Building mzQC JSON file...

✓ mzQC file saved to: demo_qc_metrics.mzQC.json
✓ Heatmap saved to: demo_qc_heatmap.png

✅ Metrics calculation complete!


## Display Metrics Table

Let's display the metrics in a formatted table.

In [ ]:
# Print formatted metrics tables
print("="*120)
print("QC METRICS TABLES")
print("="*120)
print_metrics_tables(json_output)

QC METRICS TABLES
QUALITY METRICS SUMMARY
Analyzing 3 run(s): demo_data/BSA1.mzML, demo_data/BSA2.mzML, demo_data/BSA3.mzML

=== INSTRUMENT METADATA ===
             Property                                     demo_data/BSA1.mzML                                     demo_data/BSA2.mzML                                     demo_data/BSA3.mzML
  Analyzer resolution                                                     0.0                                                     0.0                                                     0.0
Instrument model name                                         LTQ Orbitrap XL                                         LTQ Orbitrap XL                                         LTQ Orbitrap XL
         Manufacturer                                                                                                                                                                        
   Original file name                                               BSA1.mzML          

## View mzQC JSON Output

The complete mzQC format output with all metrics and metadata.

In [ ]:
# Parse and display JSON in a nicely formatted way
mzqc_data = json.loads(json_output)
display(JSON(mzqc_data, expanded=False))

<IPython.core.display.JSON object>

## Display Heatmap Visualization

Visual comparison of QC metrics across all runs.

In [ ]:
# Display the generated heatmap
if os.path.exists(output_plot):
    display(Image(filename=output_plot))
else:
    print("Heatmap not found!")

## Summary

This notebook demonstrated:

1. **Library usage**: Calling `calculate_metrics()` programmatically
2. **Data processing**: Computing 100+ QC metrics from mzML files
3. **Output formats**:
   - mzQC JSON (standard format)
   - Formatted tables
   - Heatmap visualization

### Next Steps

- Use your own mzML files by changing the `demo_file_paths` list
- Integrate into your QC pipeline
- Compare metrics across different experiments
- Set quality thresholds based on the metrics

## Clean Up (Optional)

Remove demo files and outputs if desired.

In [ ]:
# Uncomment to clean up demo files
# import shutil
# shutil.rmtree(demo_dir)
# os.remove(output_json)
# os.remove(output_plot)
# print("Cleanup complete!")